In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import yfinance as yf

 

In [19]:

def price_resampler(data, timeframe, timeframe_label):

    resampled_data = data.resample(timeframe).agg({'Open': 'first', 
                                                   'High': 'max', 
                                                   'Low': 'min', 
                                                   'Close': 'last'}).dropna()  

    resampled_data_closed = resampled_data[:-1]

    resampled_data_closed['timeframe'] = timeframe_label

    return resampled_data_closed



current_year = date.today().year
current_month = date.today().month
current_day = date.today().day

extract_start_day = str(max(current_day - 6,1)).zfill(2)
extract_start = str(current_year)+"-"+str(current_month)+"-"+extract_start_day
extract_end = date.today() + timedelta(days=10)

years = [current_year - 1, current_year]
months = range(1,13)

filenames = []
for year in years:
    for month in months:
        if (year < current_year) | (month <= current_month):
            filenames.append(str(year)+"_"+str(month).zfill(2)+".csv")

filenames = filenames[-6:]


for i in range(len(filenames)):
    file_loc = 's3://bucket-fxdata/gbpjpy/fiveminute/'+filenames[i]
    df = pd.read_csv(file_loc)
    if i == 0:
        data_5m = df.copy()
    else:
        data_5m = data_5m.append(df)

data_5m['datetime'] = pd.to_datetime(data_5m.datetime, format = '%Y/%m/%d %H:%M:%S')
data_5m.index = data_5m['datetime']
data_5m = data_5m.drop(columns=['datetime'])
data_5m['timeframe'] = '5m'

data_15m = price_resampler(data_5m, '15T', '15m')
data_30m = price_resampler(data_5m, '30T', '30m')
data_1h = price_resampler(data_5m, '1H', '1h')
data_5m_offset = data_5m.copy()
data_5m_offset.index = data_5m_offset.index + pd.DateOffset(hours=2)
data_4h = price_resampler(data_5m_offset, '4H', '4h')
data_4h.index = data_4h.index - pd.DateOffset(hours=2)

data = data_5m.append(data_15m).append(data_30m).append(data_1h).append(data_4h)

data['hour'] = data.index.hour
data['hour'] = data['hour'].astype(str).str.zfill(2)

data['minute'] = data.index.minute
data['minute'] = data['minute'].astype(str).str.zfill(2)

data['time'] = data['hour']+" "+data['minute']

data['candle_size_h'] = abs(data['Open'] - data['High'])
data['candle_size_l'] = abs(data['Open'] - data['Low'])
data['candle_size'] = data[['candle_size_h', 'candle_size_l']].max(axis=1) * 100

data = data[(data['candle_size_h'] != 0) & (data['candle_size_l'] != 0)]

data['days_since'] = (pd.to_datetime(date.today()) - data.index).days
data['days_since'] = data['days_since'] - data['days_since'].min()

data['bullish'] = np.where(data['Close'] > data['Open'], 1, 0)

data_last100 = data[data['days_since'] <= 100]

data_last100['date'] = data_last100.index

data_last100 = data_last100[['date','timeframe','hour','minute','time','candle_size','bullish','days_since']]



C:\Users\ralph\AppData\Local\Temp/ipykernel_34916/4141142069.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resampled_data_closed['timeframe'] = timeframe_label
C:\Users\ralph\AppData\Local\Temp/ipykernel_34916/4141142069.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_last100['date'] = data_last100.index


In [24]:
data4h = data[data.timeframe == '30m']

In [29]:
data4h[data4h.index >= '2022-01-12']

,Open,High,Low,Close,timeframe,hour,minute,time,candle_size_h,candle_size_l,candle_size,days_since,bullish
datetime,,,,,,,,,,,,,
2022-01-12 00:00:00,157.223999,157.302994,157.190002,157.244995,30m,00,00,00 00,0.078995,0.033997,7.899475,2,1
2022-01-12 00:30:00,157.251007,157.311996,157.139999,157.166000,30m,00,30,00 30,0.060989,0.111008,11.100769,1,0
2022-01-12 01:00:00,157.171005,157.214996,157.029999,157.059998,30m,01,00,01 00,0.043991,0.141006,14.100647,1,0
2022-01-12 01:30:00,157.065002,157.222000,157.039993,157.175003,30m,01,30,01 30,0.156998,0.025009,15.699768,1,1
2022-01-12 02:00:00,157.179001,157.408997,157.128006,157.376007,30m,02,00,02 00,0.229996,0.050995,22.999573,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13 05:00:00,157.110001,157.147003,157.037003,157.091995,30m,05,00,05 00,0.037003,0.072998,7.299805,0,0
2022-01-13 05:30:00,157.093002,157.130005,156.990005,157.052994,30m,05,30,05 30,0.037003,0.102997,10.299683,0,0
2022-01-13 06:00:00,157.059998,157.139008,157.016006,157.100998,30m,06,00,06 00,0.079010,0.043991,7.901001,0,1


In [17]:
file_loc = 's3://bucket-fxdata/gbpjpy/oneminute/2021_12 (5).csv'

In [18]:
test = pd.read_csv(file_loc)

FileNotFoundError: bucket-fxdata/gbpjpy/oneminute/2021_12 (5).csv